<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/Modeling/Trad_ML_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/Git-Thesis22-23/"

Mounted at /content/gdrive


In [3]:
import pandas as pd

df = pd.read_csv(f'{dir}Official/MFC_prepared.csv', usecols=["code_frames", "annotations"], header=0)
df.code_frames = df.code_frames.astype(int)

label_dict = {}
for x in range(15):
    label_dict[x+1] = x

label_dict
df['label'] = df.code_frames.replace(label_dict)
df = df.drop(["code_frames"], axis=1)
df

,annotations,label
0,Immigrants without HOPE need help entering col...,9
1,"But in the eyes of the law, he is an illegal i...",4
2,"Reaction to Tancredo, Lamm as predicted",14
3,"That, said the congressman, is what always hap...",12
4,"$50,000 per entry",0
...,...,...
46794,Smoking is becoming a social taboo,10
46795,Nor does it aid lawyers seeking novel ways to...,4
46796,'Ashes to Ashes',14
46797,SMOKE SCREEN IS SEEN BEHIND THE SMOKELESS,14


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.annotations.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42,
                                                  stratify=df.label.values
                                                )

In [5]:
zero_index_label = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14']

NB

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [7]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=zero_index_label))

accuracy 0.48689458689458687
              precision    recall  f1-score   support

           0       0.84      0.30      0.45       500
           1       0.00      0.00      0.00        38
           2       1.00      0.04      0.09       223
           3       0.00      0.00      0.00       178
           4       0.46      0.91      0.61      1454
           5       0.61      0.05      0.10       512
           6       0.74      0.54      0.62       756
           7       1.00      0.01      0.02       168
           8       0.93      0.21      0.35       386
           9       1.00      0.00      0.01       253
          10       0.62      0.04      0.07       423
          11       0.95      0.05      0.09       397
          12       0.43      0.94      0.59      1472
          13       0.00      0.00      0.00        61
          14       0.00      0.00      0.00       199

    accuracy                           0.49      7020
   macro avg       0.57      0.21      0.20      70

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM

In [8]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=10, tol=None)),
               ])
sgd.fit(X_train, y_train)



Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=10, random_state=42,
                               tol=None))])

In [9]:
%%time

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=zero_index_label))

accuracy 0.6307692307692307
              precision    recall  f1-score   support

           0       0.65      0.65      0.65       500
           1       0.50      0.08      0.14        38
           2       0.78      0.43      0.56       223
           3       0.59      0.24      0.34       178
           4       0.62      0.91      0.74      1454
           5       0.50      0.21      0.30       512
           6       0.66      0.75      0.70       756
           7       0.79      0.37      0.50       168
           8       0.68      0.59      0.63       386
           9       0.52      0.21      0.30       253
          10       0.61      0.27      0.38       423
          11       0.73      0.41      0.52       397
          12       0.62      0.89      0.73      1472
          13       0.58      0.11      0.19        61
          14       0.45      0.13      0.20       199

    accuracy                           0.63      7020
   macro avg       0.62      0.42      0.46      702

LR

In [17]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=5, C=1e5)),
               ])
logreg.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=100000.0, n_jobs=5))])

In [18]:
%%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=zero_index_label))

accuracy 0.6824786324786325
              precision    recall  f1-score   support

           0       0.73      0.70      0.72       500
           1       0.78      0.18      0.30        38
           2       0.74      0.61      0.67       223
           3       0.62      0.51      0.56       178
           4       0.80      0.80      0.80      1454
           5       0.41      0.57      0.48       512
           6       0.71      0.72      0.72       756
           7       0.73      0.55      0.63       168
           8       0.70      0.63      0.66       386
           9       0.50      0.47      0.48       253
          10       0.51      0.53      0.52       423
          11       0.67      0.57      0.62       397
          12       0.78      0.82      0.80      1472
          13       0.50      0.52      0.51        61
          14       0.34      0.28      0.31       199

    accuracy                           0.68      7020
   macro avg       0.63      0.57      0.58      702

In [23]:
y_pred = logreg.predict(['Immigrants without HOPE need help entering college'])
y_pred[0]+1

10

# Inference

In [27]:
import numpy as np
data_sm = pd.read_csv(f'{dir}output_gru_lstm/curated_dataset-last.csv', header=0)
data_sm['LR_framing'] = np.nan
data_sm['SVM_framing'] = np.nan
data_sm.tail()
     

,Text,Annotate,LSTM_framing,GRU_framing,LR_framing,SVM_framing
19995,“Itataas natin ang ating alert status. Magpap...,7.0,7.0,7.0,NaN,NaN
19996,(We will raise our alert status. We will depl...,8.0,8.0,8.0,NaN,NaN
19997,“Bukas o mamayang gabi ay ibababa ang heighte...,8.0,8.0,8.0,NaN,NaN
19998,(The heightened alert status will be raised e...,8.0,8.0,8.0,NaN,NaN
19999,Maranan said that the PNP will allocate its a...,8.0,6.0,9.0,NaN,NaN


In [29]:
save=0

for i, row in data_sm.iterrows():
    LR, SVM = logreg.predict([row['Text']])[0]+1, sgd.predict([row['Text']])[0]+1
    data_sm.at[i,'LR_framing'] = LR
    data_sm.at[i,'SVM_framing'] = SVM

    if(i%100==0):
        print(i)
    if (i%2000==0):
        print('saved', save)
        data_sm.to_csv(f'{dir}output_gru_lstm/curated_dataset-withTrad-{save}.csv', index=False)
        save+=1

0
saved 0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
saved 1
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
saved 2
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
saved 3
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
saved 4
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
saved 5
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
saved 6
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
saved 7
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
saved 8
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17

In [30]:
data_sm.to_csv(f'{dir}output_gru_lstm/curated_dataset-withTrad-last.csv', index=False)